In [1]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tashaphyne

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
from tashaphyne import normalize
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from nltk.corpus import stopwords
from tqdm import tqdm
import numpy as np
stop_words = list(set(stopwords.words('arabic')))

In [6]:
lst = []
folders = []
data_folder = 'data'

for folder in os.listdir(data_folder):
    for file in os.listdir(os.path.join(data_folder,folder)):
        f = open(os.path.join(data_folder,folder,file),encoding='utf-8')
        lst.append(f.read())
        folders.append(folder)

In [7]:
len(lst)

19500

In [8]:
len(folders)

19500

# preprocessing

In [9]:
clean = []
for sent in tqdm(lst):
    text = re.sub(r'[^ء-ي]',' ',sent)
    text = ' '.join([word.strip() for word in text.split() if word != ' '])
    text = ' '.join([word for word in text.split() if word not in stop_words])
    text = normalize.normalize_searchtext(text)
    clean.append(text)

100%|███████████████████████████████████████████████████████████████████████████| 19500/19500 [00:51<00:00, 376.54it/s]


In [10]:
clean[:30]

['ينظم معهد الشارقه للفنون معرضا فنيا عنوان باقه الفن وذلك الساعه السابعه اليوم مقر المعهد منطقه الفنون حي الشويهين الشارقه وتتلاقي المعرض ابداعات طالبا المنتسبين للدراسه المعهد كافه التخصصات الفنيه',
 'تقيم الفنانه ليتا كابيلوت معرضا مطلع العام المقبل دبي ويضم المعرض لوحه تركز خلالها الوجوه كابيلوت ولدت العام مجتمع غجري لام هجرتها تتجاوز الثلاثه شهور عمرها لتتكفل جدتها الغجريه بتربيتها سن الثامنه عمرها بعدما منعتها الالتحاق بالمدرسه تسليمها مابعد المياتم جدتها شكل نقطه تحول بارزه حياتها لتتدرج رويدا رويدا عالم الفن نضوج واكتشاف موهبتها ابرز لاحظه النقاد حول اعمال ليتا كابيلوت اعمالها الفنيه استطاعت تخلق عالما اكبر الصوره الفنيه الوحه هيءه كاءن يتالف الالوان الزيتيه والجبس والقماش لتستطيع اثاره الجدل بحق خلال استفزازها للخيال وهي تلتقط عوالم فتيات الشوارع والغانيات والمهرجين والموسيقيين وقد جمعت كتاب صدر دار فريدا كاهلو تعد نفسها تناول مشروعها المهم تسميه الماسي الانسانيه تسلط خلاله الضوء المرض العقلي محور تجربتها الجديده ضمن اسلوبها يعتمد الابيض والاسود وتفلسفهما خاص',
 'تتواصل الليله

## 190000 features because of memory lake

In [11]:
vectorizer = TfidfVectorizer(max_features=190000)
X = vectorizer.fit_transform(clean)
X = X.T.toarray()
df = pd.DataFrame(X, index=vectorizer.get_feature_names_out())

In [14]:
import numpy as np
def check_similarity(query, df):
    print("query:", query)
    print('')
    # Convert the query become a vector
    text = re.sub(r'[^ء-ي]',' ',query)
    text = ' '.join([word.strip() for word in text.split() if word != ' '])
    query = normalize.normalize_searchtext(text)
    query = [query]
    query_vec = vectorizer.transform(query).toarray().reshape(df.shape[0],)
    sim = {}
    for i in range(30):
        sim[i] = np.dot(df.loc[:, i].values, query_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(query_vec)

        sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)

    for k, v in sim_sorted:
        if v != 0.0:
            print("similarity score:", v)
            print(lst[k])
            print('----------------------------------')

query_sent = input('enter search query: ')
print('*******************************************')

check_similarity(query_sent, df)

enter search query: جورج أورويل
*******************************************
query: جورج أورويل

similarity score: 0.1301016689554909
اختارت صحيفة «التايمز» جورج أورويل ليحل في المرتبة الثانية في قائمة تضم أعظم خمسين كاتباً بريطانياً، منذ عام 1945، وقد اشتهر هذا الكاتب البريطاني بمعاداته للفاشية، وباعتناقه أفكار الاشتراكية الديمقراطية، وكانت روايتاه «1984» و«مزرعة الحيوان» في قوائم الأكثر مبيعاً في القرن العشرين، برغم منعهما لعدة سنوات في أكثر من بلد في أنحاء العالم، وقد أصدرت دار آفاق للنشر والتوزيع طبعة جديدة من رواية «مزرعة الحيوان»، وهي رواية مجازية أبطالها من الحيوانات الذين يمثلون أنماطاً من بشر نعرفهم جيداً، ويمثل عالمهم في حظيرتهم واقعاً في دول بعينها، إنها رواية ثورة غير مكتملة.كما أن مجلة «تايم» اختارت تلك الرواية لتكون واحدة من أفضل مئة رواية بالإنجليزية من عام 1923 إلى عام 2005، وجاء ترتيب الرواية 31 في قائمة أفضل روايات القرن العشرين التي أعدتها دار النشر الأمريكية الشهيرة «مودرن ليبراري»، كما ضمنت الرواية في مجموعة «ذخائر كتب العالم الغربي».طبعة دار آفاق اعتمدت على أقدم تر